In [1]:
!pip install pandahouse
!pip install swifter

  Preparing metadata (setup.py) ... done
  Created wheel for pandahouse: filename=pandahouse-0.2.7-py2.py3-none-any.whl size=5904 sha256=945fbd377e14c07777fadd2c9ee6232bffc3aa02e3ca1d2cc501a2a96867895e
  Stored in directory: /root/.cache/pip/wheels/04/81/63/6896730711da10812121737bb505c6a8993800e99d39432522
Successfully built pandahouse
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16507 sha256=4e82acfaa503a3eb694c010bfd3b9b9747746d3b1799b257e95fa73a497a5807
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter


In [1]:
import pandas as pd
import numpy as np
import pandahouse
import swifter
import seaborn as sns

# Сравнить retention по источникам траффика
Математически самым простым вариантом видится расчёт retention с помощью churn:

$retention = 1-churn\_rate$

$churn\_rate = \frac{Ушедшие \; пользователи}{Все \; пользователи}$

При этом возникает вопрос кого считать ушедшим пользоватлем:
- пользователя, который последний раз заходил $n$ дней назад

### Определение приемлемого времени для записи пользователя в отток

In [25]:
connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20240320'
}

In [110]:
# Запрос для определения удачности порога, при котором пользователь записывается в отток
n = 7  # Порог в днях для того, чтобы считать пользователя оттёкшим (отток при date_break>=n)

db = 'simulator_20240320'
q = f"""
WITH
  t_user_activity_dates AS (
    -- Дни, когда активничал пользователь
    SELECT DISTINCT
      user_id,
      DATE(time) AS date
    FROM {db}.feed_actions
  ),
  t_is_final_churn AS (
    -- Считать ли пользователя оттёкшим по последнему временому перерыву
    SELECT
      user_id,
      MAX(date) AS last_date,
      DATE(now()) - last_date AS date_break,
      date_break >= {n} AS is_final_churn
    FROM t_user_activity_dates
    GROUP BY user_id
  ),
  t_prev_dates AS (
    -- Даты предыдущей активности пользователей
    SELECT
      user_id,
      date,
      MIN(date) OVER (
        PARTITION BY user_id
        ORDER BY date
        ROWS BETWEEN 1 PRECEDING AND CURRENT ROW
      ) AS prev_date
    FROM t_user_activity_dates
  ),
  t_user_date_breaks AS (
    -- Временные перерывы пользователей (не включая последний)
    SELECT
      user_id,
      date,
      prev_date,
      date - prev_date AS date_break,
      date_break >= {n} AS is_churn
    FROM t_prev_dates
  ),
  t_final_churn_users AS (
    SELECT DISTINCT user_id
    FROM t_is_final_churn
    WHERE is_final_churn = 1
  ),
  t_not_final_churn_users AS (
    SELECT DISTINCT user_id
    FROM t_user_date_breaks
    WHERE is_churn = 1
  ),
  t_all_churn AS (
    -- Все пользователи, которых записали в отток
    SELECT DISTINCT user_id
    FROM
    (
      (
        SELECT *
        FROM t_final_churn_users
      )
      UNION ALL
      (
        SELECT *
        FROM t_not_final_churn_users
      )
    )
  ),
  t_real_churn AS (
    SELECT *
    FROM t_final_churn_users
      LEFT ANTI JOIN t_not_final_churn_users
        USING user_id
  ),
  t_churns AS (
    SELECT
      {n} AS churn_break_n,
      all_churn AS churn,
      real_churn / all_churn AS churn_precision
    FROM (
      SELECT COUNT(*) AS real_churn
      FROM t_real_churn
    ) AS l
      CROSS JOIN (
          SELECT COUNT(*) AS all_churn
          FROM t_all_churn
        ) AS r
  )

SELECT *
FROM t_churns
"""

df = pandahouse.read_clickhouse(q, connection=connection)
df

,churn_break_n,churn,churn_precision
0,7,79195,0.211667
